<a href="https://colab.research.google.com/github/tarupathak30/Deep-Learning-/blob/main/ann_keras/hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving diabetes.csv to diabetes.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/content/diabetes.csv')

In [4]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [6]:
x = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [8]:
x = scaler.fit_transform(x)


In [9]:
x.shape

(768, 8)

In [10]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [90]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Input, Dropout

In [12]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(8, )),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.4328 - loss: 0.7456 - val_accuracy: 0.5455 - val_loss: 0.6925
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6209 - loss: 0.6648 - val_accuracy: 0.7013 - val_loss: 0.6313
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7114 - loss: 0.6206 - val_accuracy: 0.7468 - val_loss: 0.5867
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7408 - loss: 0.5934 - val_accuracy: 0.7792 - val_loss: 0.5565
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7524 - loss: 0.5558 - val_accuracy: 0.7727 - val_loss: 0.5330
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7622 - loss: 0.5212 - val_accuracy: 0.7792 - val_loss: 0.5161
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7309 - loss: 0.5392 - val_accuracy: 0.7792 - val_loss: 0.5057
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7896 - loss: 0.4949 - val_accuracy: 0.7727 - val_los

1. how to select appropriate optimizer
2. no. of node in a layer
3. how to select no. of layers
4. all in all one model

In [14]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.1 MB/s eta 0:00:00


In [15]:
import keras_tuner as kt

In [16]:
def build_model(hp):
  model = Sequential([
      Dense(32, activation='relu', input_dim=8),
      Dense(1, activation='sigmoid')
  ])

  optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adam', 'adadelta'])

  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [17]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5)

In [18]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 4 Complete [00h 00m 05s]
val_accuracy: 0.7467532753944397

Best val_accuracy So Far: 0.7467532753944397
Total elapsed time: 00h 00m 19s


In [19]:
best_model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [20]:
best_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [22]:
best_model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7212 - loss: 0.5356 - val_accuracy: 0.7597 - val_loss: 0.5153
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7425 - loss: 0.5120 - val_accuracy: 0.7662 - val_loss: 0.5052
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7533 - loss: 0.5203 - val_accuracy: 0.7597 - val_loss: 0.4981
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7356 - loss: 0.5069 - val_accuracy: 0.7662 - val_loss: 0.4942
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7517 - loss: 0.4876 - val_accuracy: 0.7792 - val_loss: 0.4910
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7774 - loss: 0.4728 - val_accuracy: 0.7792 - val_loss: 0.4889
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7698 - loss: 0.4718 - val_accuracy: 0.7792 - val_loss: 0.4869
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7692 - loss: 0.4812 - val_accuracy: 0.

In [23]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units', min_value=8, max_value=128, step=8)

  model.add(Dense(units=units, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [24]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='mydir')

In [25]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test), verbose=1)

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 13s


In [26]:
tuner.get_best_hyperparameters()[0].values

{'units': 128}

In [27]:
model = tuner.get_best_models(num_models=1)[0]

In [28]:
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7807 - loss: 0.4681 - val_accuracy: 0.8117 - val_loss: 0.4636
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7693 - loss: 0.4646 - val_accuracy: 0.8117 - val_loss: 0.4611
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7627 - loss: 0.4606 - val_accuracy: 0.7987 - val_loss: 0.4624
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7777 - loss: 0.4401 - val_accuracy: 0.7922 - val_loss: 0.4616
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7831 - loss: 0.4352 - val_accuracy: 0.7922 - val_loss: 0.4615
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7792 - loss: 0.4342 - val_accuracy: 0.7987 - val_loss: 0.4628
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7702 - loss: 0.4571 - val_accuracy: 0.7987 - val_loss: 0.4613
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7815 - loss: 0.4313 - val_accuracy: 0.79

In [39]:
def build_model(hp):
  model = Sequential()
  model.add(Input(shape=(8, )))
  model.add(Dense(72, activation='relu'))

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(72, activation='relu'))

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

  return model

In [40]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='num_layers')

In [41]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 12s


In [42]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 3}

In [43]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [49]:
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.7886 - loss: 0.4693 - val_accuracy: 0.7922 - val_loss: 0.4788
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7834 - loss: 0.4291 - val_accuracy: 0.8052 - val_loss: 0.4643
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8039 - loss: 0.4314 - val_accuracy: 0.8117 - val_loss: 0.4628
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8062 - loss: 0.4218 - val_accuracy: 0.7662 - val_loss: 0.5079
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8092 - loss: 0.4036 - val_accuracy: 0.7792 - val_loss: 0.4826
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8058 - loss: 0.4001 - val_accuracy: 0.7987 - val_loss: 0.4763
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8228 - loss: 0.3854 - val_accuracy: 0.7727 - val_loss: 0.5060
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8037 - loss: 0.4145 - val_accuracy: 0.78

In [91]:
def build_model(hp):
  model = Sequential()
  model.add(Input(shape=(8, )))
  counter = 0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):

    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    else:
      model.add(
          Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid']))
      )
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])))
    counter+=1

  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer=hp.Choice('optimizer',
                                    values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                                    loss='binary_crossentropy',
                                    metrics=['accuracy'])
  return model

In [92]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='mydir',
                        project_name='experiment_layers_activation_nodes',
                        overwrite=True)

In [93]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 3 Complete [00h 00m 06s]
val_accuracy: 0.7792207598686218

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 16s


In [94]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 128,
 'activation0': 'tanh',
 'dropout0': 0.4,
 'optimizer': 'adam',
 'units1': 40,
 'activation1': 'sigmoid',
 'dropout1': 0.5,
 'units2': 112,
 'activation2': 'relu',
 'dropout2': 0.4,
 'units3': 32,
 'activation3': 'tanh',
 'dropout3': 0.4,
 'units4': 56,
 'activation4': 'sigmoid',
 'dropout4': 0.6,
 'units5': 120,
 'activation5': 'relu',
 'dropout5': 0.7,
 'units6': 64,
 'activation6': 'relu',
 'dropout6': 0.8,
 'units7': 8,
 'activation7': 'tanh',
 'dropout7': 0.7}

In [95]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [96]:
tuner.results_summary()

Results summary
Results in mydir/experiment_layers_activation_nodes
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 4
units0: 128
activation0: tanh
dropout0: 0.4
optimizer: adam
units1: 40
activation1: sigmoid
dropout1: 0.5
units2: 112
activation2: relu
dropout2: 0.4
units3: 32
activation3: tanh
dropout3: 0.4
units4: 56
activation4: sigmoid
dropout4: 0.6
units5: 120
activation5: relu
dropout5: 0.7
units6: 64
activation6: relu
dropout6: 0.8
units7: 8
activation7: tanh
dropout7: 0.7
Score: 0.7792207598686218

Trial 0 summary
Hyperparameters:
num_layers: 8
units0: 24
activation0: sigmoid
dropout0: 0.6
optimizer: sgd
units1: 8
activation1: relu
dropout1: 0.1
units2: 8
activation2: relu
dropout2: 0.1
units3: 8
activation3: relu
dropout3: 0.1
units4: 8
activation4: relu
dropout4: 0.1
units5: 8
activation5: relu
dropout5: 0.1
units6: 8
activation6: relu
dropout6: 0.1
units7: 8
activation7: relu
dropout7: 0.1
Score: 0.6428571

In [97]:
model.fit(x_train, y_train, epochs=200, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.7211 - loss: 0.5458 - val_accuracy: 0.7857 - val_loss: 0.4792
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7296 - loss: 0.5352 - val_accuracy: 0.8052 - val_loss: 0.4715
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6860 - loss: 0.5610 - val_accuracy: 0.8052 - val_loss: 0.4711
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6979 - loss: 0.5597 - val_accuracy: 0.7792 - val_loss: 0.4740
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7097 - loss: 0.5545 - val_accuracy: 0.7922 - val_loss: 0.4721
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7269 - loss: 0.5466 - val_accuracy: 0.7987 - val_loss: 0.4735
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7194 - loss: 0.5188 - val_accuracy: 0.7857 - val_loss: 0.4802
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7614 - loss: 0.4939 - val_accuracy: 0.7